In [92]:
from rasa.nlu.model import Interpreter, Metadata
import pandas as pd
import datetime
import numpy as np
import word2number

DATASET_FILE_NAME = "edi_air_quality.csv"
DATASET_PATH = f"data/{DATASET_FILE_NAME}"
dataset = pd.read_csv(DATASET_PATH)

for column in dataset.columns:
    if column != "date":
        dataset[column] = pd.to_numeric(dataset[column], errors='coerce')

utterance = u"was the air quality great the last 2 weeks"
model_name = "nlu-20200209-185009"
dataset_name = "nlu"
model = f"models/{model_name}/{dataset_name}"




# loading the model from one directory or zip file
interpreter = Interpreter.load(model)

# parsing the utterance
interpretation = interpreter.parse(utterance)

# printing the parsed output
interpretation

INFO:tensorflow:Restoring parameters from models/nlu-20200209-185009/nlu/component_6_EmbeddingIntentClassifier.ckpt


INFO:tensorflow:Restoring parameters from models/nlu-20200209-185009/nlu/component_6_EmbeddingIntentClassifier.ckpt


{'intent': {'name': 'air_quality_historical',
  'confidence': 0.9964810609817505},
 'entities': [{'start': 20,
   'end': 25,
   'value': ' good',
   'entity': 'good',
   'confidence': 0.5198660373982853,
   'extractor': 'CRFEntityExtractor',
   'processors': ['EntitySynonymMapper']},
  {'start': 30,
   'end': 34,
   'value': 'last',
   'entity': 'hierarchy_number',
   'confidence': 0.963261152352626,
   'extractor': 'CRFEntityExtractor'},
  {'start': 35,
   'end': 36,
   'value': '2',
   'entity': 'number',
   'confidence': 0.9779956776019908,
   'extractor': 'CRFEntityExtractor'},
  {'start': 37,
   'end': 42,
   'value': 'weeks',
   'entity': 'time_measures',
   'confidence': 0.9833818456716202,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'air_quality_historical',
   'confidence': 0.9964810609817505},
  {'name': 'deny', 'confidence': 0.0012664454989135265},
  {'name': 'greet', 'confidence': 0.0006883703172206879},
  {'name': 'affirm', 'confidence': 0.00051935

In [93]:
def row_is_between_dates(row, low_date, up_date):

    date_row = row["date"].split("/")
    true_date_row = datetime.date(int(date_row[0]), int(date_row[1]), int(date_row[2]))
    
    return true_date_row <= up_date and true_date_row > low_date

In [94]:
def get_response_from_rasa_interpretation(interpretation):
    intent = interpretation['intent']['name']

    returned_response = "I did not get that. Could you repeat ?"

    if intent == "air_quality_today":
        r = requests.get(url=API_URL)
        response = r.json()
        returned_response = f"""
        Today's air is as follows:
            - h: {response['data']['h']['v']}
            - no2: {response['data']['no2']['v']}
            - o3: {response['data']['o3']['v']}
            - p: {response['data']['p']['v']}
            - pm10: {response['data']['pm10']['v']}
            - pm25: {response['data']['pm25']['v']}
            - so2: {response['data']['so2']['v']}
        """
        
    elif intent == "air_quality_historical":
        
        returned_response = ""
        
        number = None
        hierarchy_number = None
        time_measures = None
        for entity in interpretation['entities']:
            
            if entity["entity"] == "hierarchy_number":
                hierarchy_number = entity["value"]
                
            elif entity["entity"] == "time_measures":
                time_measures = entity["value"]
                
            elif entity["entity"] == "number":
                number = entity["value"]
        
        if number is None and hierarchy_number is None and time_measures is None:
            returned_response = "Query not fully understood, returning last week data by default:"
            hierarchy_number = "last"
            time_measures = "week"
            
        # count the number of unit of time measures
        counter = 1
        # convert the unit of time measures as days
        time_measure_factor = 1
        
        if (time_measures[0:4] == "week"):
            time_measure_factor = 7
        elif (time_measures[0:5] == "month"):
            time_measure_factor = 30
        elif (time_measures[0:4] == "year"):
            time_measure_factor = 365
    
        # e.g. last week
        if (hierarchy_number == "last" and number is None):
            counter = 1    
        # e.g last 2 weeks
        elif (hierarchy_number == "last" and number is not None):
            try:
                counter = int(number)
            except:
                counter = w2n.word_to_num(number)
        elif (number is not None):
            try:
                counter = number
            except:
                counter = w2n.word_to_num(number)
        # e.g one month ago (we then check the date one month ago and move 7 days ahead)
        if (hierarchy_number != "last"):
            
            d_up = datetime.date.today() - datetime.timedelta(days=counter*time_measure_factor) + datetime.timedelta(days = 7)
    
        else:
    
            d_up = datetime.date.today() # to switch for interval of time
        
        d_down = d_up - datetime.timedelta(days=counter*time_measure_factor)
        
        rows_of_dataset = dataset[dataset.apply(lambda x: row_is_between_dates(x, low_date = d_down, up_date = d_up), axis=1)]
        
        returned_response+=f"""
        \n
        Air quality of the last {counter*time_measure_factor} days was like this:
        
        """
        
        for column in dataset.columns:
            if column != "date":
                returned_response += f"""
                    - The average value of {column} is {dataset[column].mean()}
                """
        
    return returned_response

In [95]:
get_response_from_rasa_interpretation(interpretation)

'\n        \n\n        Air quality of the last 14 days was like this:\n        \n        \n                    - The average value of  pm25 is 26.773117736230418\n                \n                    - The average value of  pm10 is 10.559270516717325\n                \n                    - The average value of  o3 is 21.895566502463055\n                \n                    - The average value of  no2 is 8.139246778989099\n                \n                    - The average value of  so2 is 3.616358839050132\n                \n                    - The average value of  co is 3.2099927588703836\n                '

In [91]:
datetime.datetime.now()

datetime.datetime(2020, 2, 9, 21, 20, 44, 630955)